In [1]:
Pkg.add("POMDPs")
using POMDPs
POMDPs.add("POMDPToolbox")
using POMDPToolbox # for WeightVec, HistoryRecorder, etc

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of POMDPs
INFO: Use `Pkg.update()` to get the latest versions of your packages


Package already installed


INFO: Cloning POMDPToolbox from https://github.com/JuliaPOMDP/POMDPToolbox.jl


In [2]:
importall POMDPs
include("helpers.jl");

In [3]:
plot_chain(4, fill=Dict(4=>"green"))

In [4]:
type ChainMDP <: MDP{Int, Bool}
    len::Int
    p_success::Float64
    discount::Float64
end
discount(mdp::ChainMDP) = mdp.discount
# left is false, right is true

discount (generic function with 2 methods)

In [5]:
create_transition_distribution(mdp::ChainMDP) = Array(Float64, mdp.len)
pdf(d::Vector{Float64}, s::Int) = d[s]
rand(rng::AbstractRNG, d::Vector{Float64}) = sample(rng, WeightVec(d))
rand(rng::AbstractRNG, d::Vector{Float64}, s::Int) = sample(rng, WeightVec(d))

rand (generic function with 123 methods)

In [6]:
function transition(mdp::ChainMDP, s::Int, a::Bool, d::Vector{Float64}=create_transition_distribution(mdp))
    fill!(d, 0.0)
    if a == true
        forward = s+1
        back = s-1
    else
        forward = s-1
        back = s+1
    end
    forward = clamp(forward, 1, mdp.len)
    back = clamp(forward, 1, mdp.len)
    d[forward] = mdp.p_success
    d[back] = 1.0 - mdp.p_success
    return d
end

transition (generic function with 4 methods)

In [7]:
function reward(mdp::ChainMDP, s::Int, a::Bool)
    if s == mdp.len
        return 1.0
    else
        return 0.0
    end 
end

reward (generic function with 3 methods)

In [8]:
rec = HistoryRecorder(eps = 0.01)

POMDPToolbox.HistoryRecorder(MersenneTwister(UInt32[0x856e6eb3],Base.dSFMT.DSFMT_state(Int32[197681839,1073390329,385345437,1072970257,1538990630,1073587903,-1556803565,1072775732,2037099528,1073618531  …  -1764165109,1073392391,-65317283,1073351731,-1878779743,-1776100276,-1498850280,1308151426,382,0]),[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],382),Any[],Any[],Any[],Any[],Nullable{Exception}(),Nullable{Any}(),false,Nullable{Any}(),Nullable{Any}(0.01),Nullable{Any}(),Nullable{Integer}())

In [9]:
mdp = ChainMDP(4, 0.7, 0.9)

ChainMDP(4,0.7,0.9)

In [14]:
policy = VectorPolicy(mdp, fill(true, 4))

POMDPToolbox.VectorPolicy{Int64,Bool}(ChainMDP(4,0.7,0.9),Bool[true,true,true,true])

In [15]:
simulate(rec, mdp, policy, 1)

7.193022627021248

In [16]:
rec.state_hist

45-element Array{Int64,1}:
 1
 2
 3
 4
 4
 4
 4
 4
 4
 4
 4
 4
 4
 ⋮
 4
 4
 4
 4
 4
 4
 4
 4
 4
 4
 4
 4